In [1]:
import numpy as np
import torch
from ucimlrepo import fetch_ucirepo 
import torch.nn as nn     
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import copy
import main
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets 
y["class"]=y["class"].replace("Iris-setosa",0.0)
y["class"]=y["class"].replace("Iris-versicolor",1.0)
y["class"]=y["class"].replace("Iris-virginica",2.0)
# metadata 
X=X.values
y=y.values
y=y.squeeze(axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
mean = X_train.mean(axis=0)  # Mean of training set
std = X_train.std(axis=0)  # Standard deviation of training set

# Avoid division by zero for constant features
std[std == 0] = 1

# Apply mean normalization (centering and scaling) to training and test sets


C:\Users\Sebas\AppData\Local\Temp\ipykernel_2832\1896799476.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["class"]=y["class"].replace("Iris-setosa",0.0)
C:\Users\Sebas\AppData\Local\Temp\ipykernel_2832\1896799476.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["class"]=y["class"].replace("Iris-versicolor",1.0)
C:\Users\Sebas\AppData\Local\Temp\ipykernel_2832\1896799476.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the

In [3]:
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)
#print(iris.metadata) 
  
# variable information 
#print(iris.variables) 
class Model(nn.Module):
    def __init__(self,in_features=4,h1=16,out=3):
        super().__init__()
        self.fc1=nn.Linear(in_features,h1,bias=False)
        self.out=nn.Linear(h1,out)
    def forward(self,x):
        x=F.relu(self.fc1(x))
        #x=F.relu(self.fc2(x))
        x=self.out(x)
        return x
def iris_training(h1):
    model=Model(h1=h1)
    W_untrained=copy.deepcopy(next(model.parameters())).detach().numpy().astype(float)
    #print(W_untrained)
    criterion=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
    epochs=100
    losses=[]
    for i in range(epochs):
        y_pred=model.forward(X_train)
        loss=criterion(y_pred,y_train)
        losses.append(loss.detach().numpy())
        #if i % 10 == 0:
           # print(f'Epoch: {i} and loss: {loss}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    W_trained=next(model.parameters()).detach().numpy().astype(float)
    return W_untrained,W_trained,loss.detach().numpy()
#plt.plot(range(epochs),losses)

for m in range(16,35):
    print(f"start {m}")
    losses=[]
    untrained=[]
    trained=[]
    for i in range(1000):
        res=iris_training(m)
        losses.append(res[2])
        untrained.append(res[0])
        trained.append(res[1])
    losses=np.array(losses)
    print(np.mean(losses))
    res=0
    for W in untrained:
    # print(W.T.shape)
        if not main.findCell(np.array([]),W):
            res+=1
    res
    train=0
    for W in trained:
        if not main.findCell(np.array([]), W):
            train+=1
    print(f"m={m} untrained= {res},trained = {train}")

# For m=18 314 untrained 288 trained error 0.14 
# 324-308 0.14
#For m=16 112- 102 error 0.15
# 0.155  114 - 84
# For m=17 0.149 236- 196
# 0.15 225-192
#m=18 untrained= 336,trained = 305 0.14248206
#m=19 untrained= 476,trained = 448, 0.13926487
#m=20 untrained= 551,trained = 541,  0.13288392
#m=21 untrained= 661,trained = 670, 0.1308894
#m=22 untrained= 771,trained = 774, 0.12922817
# m=23 untrained= 837,trained = 820, 0.12744117
#m=24 untrained= 887,trained = 875, 0.12315167

start 16
0.14855391
m=16 untrained= 119,trained = 109
start 17
0.14388794
m=17 untrained= 202,trained = 202
start 18
0.13971537
m=18 untrained= 354,trained = 329
start 19
0.13416904
m=19 untrained= 493,trained = 486
start 20
0.13159996
m=20 untrained= 546,trained = 557
start 21
0.1276451
m=21 untrained= 669,trained = 652
start 22
0.12512916
m=22 untrained= 738,trained = 732
start 23
0.121839724


KeyboardInterrupt: 

In [ ]:

X_train_numpy = (X_train - mean) / std
X_test_numpy = (X_test - mean) / std
X_train=torch.FloatTensor(X_train_numpy)
X_test=torch.FloatTensor(X_test_numpy)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)
class Model(nn.Module):
    def __init__(self,in_features=4,h1=16,out=3):
        super().__init__()
        self.fc1=nn.Linear(in_features,h1,bias=False)
        #self.fc2=nn.Linear(h1,h2)
        self.out=nn.Linear(h1,out)
    def forward(self,x):
        x=F.relu(self.fc1(x))
        #x=F.relu(self.fc2(x))
        x=self.out(x)
        return x
def iris_training(h1):
    model=Model(h1=h1)
    W_untrained=copy.deepcopy(next(model.parameters())).detach().numpy().astype(float)
    #print(W_untrained)
    criterion=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
    epochs=100
    losses=[]
    for i in range(epochs):
        y_pred=model.forward(X_train)
        loss=criterion(y_pred,y_train)
        losses.append(loss.detach().numpy())
        #if i % 10 == 0:
           # print(f'Epoch: {i} and loss: {loss}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    W_trained=next(model.parameters()).detach().numpy().astype(float)
    return W_untrained,W_trained,loss.detach().numpy()
#plt.plot(range(epochs),losses)
total_untrained=[]
total_trained=[]
total_losses=[]
res_noninj_points_train=[]
res_noninj_points_untrain=[]
for m in range(14,35):
    print(f"start {m}")
    losses=[]
    untrained=[]
    trained=[]
    for i in range(500):
        res=iris_training(m)
        losses.append(res[2])
        untrained.append(res[0])
        trained.append(res[1])
    losses=np.array(losses)
    total_losses.append(losses)
    print(np.mean(losses))
    res=0
    noninj_untrain=0

    for W in untrained:
    # print(W.T.shape)
        if not main.findCell(np.array([]),W):
            res+=2

    train=0

    for W in trained:

        non_inj_count=0
        if not main.findCell(np.array([]), W):
            train+=2

        ##

    total_untrained.append(res)
    total_trained.append(train)

    print(f"m={m} untrained= {res},trained = {train}" )
print(total_untrained,total_trained,total_losses)



start 14
0.1643767
m=14 untrained= 6,trained = 6
start 15
0.16182674
m=15 untrained= 44,trained = 20
start 16
0.16108026
m=16 untrained= 102,trained = 70
start 17
0.15832007
m=17 untrained= 240,trained = 132
start 18
0.15699334
m=18 untrained= 322,trained = 204
start 19
0.15476213
m=19 untrained= 464,trained = 264
start 20
0.15281819


Exception ignored in: <function Cplex.__del__ at 0x000001609E177250>
Traceback (most recent call last):
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\__init__.py", line 925, in __del__
    self.end()
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\__init__.py", line 921, in end
    self._env._end()
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\_internal\__init__.py", line 153, in _end
    self._delete_stream(chnl_idx)
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\_internal\__init__.py", line 248, in _delete_stream
    self._streams[which_channel]._end()
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\_internal\_ostream.py", line 88, in _end
    self.flush()
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\_internal\_ostream.py", line 148, in flush
   